In [5]:
# download the tiny Trivial Q/A dataset from Google drive
#
!gdown --folder https://drive.google.com/drive/folders/1cLjPppGCbjL6w31F1l70Q8qtzL-8pRUb?usp=share_link 2> /dev/null

Processing file 1aJNW3CKrwduxL35tBIya6TbaoNWWQc8Y answers.txt
Processing file 11KhbiY23sdpJkqzkIQQe4jLycPns6MdI prompts.txt
Processing file 1iUbTcBJEHSbA9R9OfMwKr0Lvt5S4Hbeh train.txt


In [6]:
# display the first few lines of training text, containing question and answer pairs

!cat tinyQA/train.txt | head -n 10


what was pierce brosnan's first outing as 007 [ goldeneye ]
the 02 arena is in which london borough [ greenwich ]
who wrote the 1956 novel '101 dalmatians' [ dodie smith ]
which band's first top ten single was the 10538 overture in 1972 [ electric light orchestra ]
the 1999 film 10 things i hate about you is based on which shakespeare play [ the taming of the shrew ]
the film `10 things i hate about you` is based on which shakespeare play [ the taming of the shrew ]
the film '10 things i hate about you', was inspired by which of shakespeare's plays [ the taming of the shrew ]
who directed the 2010 film 127 hours [ danny boyle ]
ciara had a hit with 1,2 step featuring which other artist [ missy elliot ]
which film director won the oscar for best picture for the film 12 years a slave in 2013 [ steve mcqueen ]
in which 2006 film does mark wahlberg try out for a university american football team [ invincible ]
'the black and gold' is a nickname of which american football team [ pittsburgh 

In [ ]:
# display the six questions as prompts

!cat tinyQA/prompts.txt

which football club did alan sugar own [
 'the black and gold' is a nickname of which american football team [ 
alex band and aaron kamin make up which band [
the aberdare mountains are in which african country [
which notable leader won the 2009 nobel peace prize [
who commanded the confederate army of northern virginia during the american civil war [ 


In [ ]:
# display the questions and answers for all prompts

!cat tinyQA/answers.txt

which football club did alan sugar own [ tottenham hotspur f.c. ]
'the black and gold' is a nickname of which american football team [ pittsburgh steelers ]
alex band and aaron kamin make up which band [ the calling ]
the aberdare mountains are in which african country [ kenya ]
which notable leader won the 2009 nobel peace prize [ barack obama ]
who commanded the confederate army of northern virginia during the american civil war [ robert e. lee ]


In [7]:
# load text file as a character string; build char-level vocabulary, encoder and decoder
#  (borrowed from nanoGPT, https://github.com/karpathy/nanoGPT)

train_txt_file = 'tinyQA/train.txt'

with open(train_txt_file, 'r') as f:
    data = f.read()
print(f"length of dataset in characters: {len(data):,}")

# get all the unique characters that occur in this text
chars = sorted(list(set(data)))
vocab_size = len(chars)
print("all the unique characters:", ''.join(chars))
print(f"vocab size: {vocab_size:,}")

# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
def encode(s):
    return [stoi[c] for c in s] # encoder: take a string, output a list of integers
def decode(l):
    return ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# create the train and test splits
n = len(data)
train_data = data[:int(n*0.9)]
val_data = data[int(n*0.9):]

# encode both to integers
train_ids = encode(train_data)
val_ids = encode(val_data)
print(f"train has {len(train_ids):,} tokens")
print(f"val has {len(val_ids):,} tokens")

length of dataset in characters: 98,391
all the unique characters: 
 !#$%&'()*,-./0123456789:;[]_`abcdefghijklmnopqrstuvwxyz
vocab size: 57
train has 88,551 tokens
val has 9,840 tokens


In [8]:
# Define all parts in a GPT model, such as attention, MLP, LN modules
#  (adapted from nanoGPT, https://github.com/karpathy/nanoGPT)

import math
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F

class config():
  def __init__(self, batch_size=10, n_layer=12, n_head=12, n_embd=768, block_size=1024, vocab_size=50304, causal=True, device='cpu'):
    assert n_embd %  n_head == 0
    self.batch_size = batch_size
    self.n_embd = n_embd
    self.block_size = block_size
    self.n_head = n_head
    self.causal = causal
    self.device = device
    self.n_layer = n_layer
    self.vocab_size = vocab_size

class CausalSelfAttention(nn.Module):

    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        # key, query, value projections for all heads, but in a batch
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=False)
        # output projection
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=False)

        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.register_buffer("mask", torch.tril(torch.ones(config.block_size, config.block_size)).transpose(0,1)
                            .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size() # batch size, sequence length, embedding dimensionality (n_embd)

        # calculate query, key, values for all heads in batch and move head forward to be the batch dim
        q, k ,v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, C // self.n_head, self.n_head).transpose(1, 3) # (B, nh, T, hs)
        q = q.view(B, T, C // self.n_head, self.n_head).transpose(1, 3) # (B, nh, T, hs)
        v = v.view(B, T, C // self.n_head, self.n_head).transpose(1, 3) # (B, nh, T, hs)

        att = (q.transpose(-2, -1) @ k) * (1.0 / math.sqrt(q.size(-2)))
        att = att.masked_fill(self.mask[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-2)
        y = v @ att  # (B, nh, T, T) x (B, nh, T, hs) -> (B, nh, T, hs)
        y = y.transpose(1, 3).contiguous().view(B, T, C) # re-assemble all head outputs side by side
        # output projection
        y = self.c_proj(y)
        return y

class MLP(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.c_fc    = nn.Linear(config.n_embd, 4 * config.n_embd, bias=False)
        self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=False)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.c_fc(x)
        x = self.relu(x)
        x = self.c_proj(x)
        return x

class Block(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.ln_1 = nn.LayerNorm([config.n_embd])
        self.attn = CausalSelfAttention(config)
        self.ln_2 = nn.LayerNorm([config.n_embd])

        self.mlp = MLP(config)

    def forward(self, x):
        x = x + self.attn(self.ln_1(x))
        x = x + self.mlp(self.ln_2(x))
        return x

class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.vocab_size is not None
        assert config.block_size is not None
        self.config = config

        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.block_size, config.n_embd),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm([config.n_embd]),
        ))
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # with weight tying when using torch.compile() some warnings get generated:
        self.transformer.wte.weight = self.lm_head.weight # https://paperswithcode.com/method/weight-tying

        # init all weights
        self.apply(self._init_weights)
        # apply special scaled init to the residual projections, per GPT-2 paper
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02/math.sqrt(2 * config.n_layer))

        # report number of parameters
        print("number of parameters: %.2fM" % (self.get_num_params()/1e6,))

    def get_num_params(self, non_embedding=True):
        # Return the number of parameters in the model.
        n_params = sum(p.numel() for p in self.parameters())
        if non_embedding:
            n_params -= self.transformer.wpe.weight.numel()
        return n_params

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size, f"Cannot forward sequence of length {t}, block size is only {self.config.block_size}"
        pos = torch.arange(0, t, dtype=torch.long, device=device).unsqueeze(0) # shape (1, t)

        # forward the GPT model itself
        tok_emb = self.transformer.wte(idx) # token embeddings of shape (b, t, n_embd)
        pos_emb = self.transformer.wpe(pos) # position embeddings of shape (1, t, n_embd)
        x = tok_emb + pos_emb
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)

        if targets is not None:
            # if we are given some desired targets also calculate the loss
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
        else:
            # inference-time mini-optimization: only forward the lm_head on the very last position
            logits = self.lm_head(x[:, [-1], :]) # note: using list [-1] to preserve the time dim
            loss = None

        return logits, loss

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):

        for _ in range(max_new_tokens):
            # if the sequence context is growing too long we must crop it at block_size
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            # forward the model to get the logits for the index in the sequence
            logits, _ = self(idx_cond)
            # pluck the logits at the final step and scale by desired temperature
            logits = logits[:, -1, :] / temperature
            # optionally crop the logits to only the top k options
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            # apply softmax to convert logits to (normalized) probabilities
            probs = F.softmax(logits, dim=-1)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)

            # append sampled index to the running sequence and continue
            idx = torch.cat((idx, idx_next), dim=1)

        return idx


In [9]:
# training script to learn GPT models based on all given hyper-parameters
#  (adapted from nanoGPT, https://github.com/karpathy/nanoGPT)

import os
import time
import math

import numpy as np
import torch

# -----------------------------------------------------------------------------
# default config values
# I/O

######### model size ##########
n_layer = 3      # num of layers
n_head = 8       # num of attn heads per layer
head_size = 96   # size of each attn head
###############################

###### training hyperparameters ######
learning_rate = 6e-4   # max learning rate
max_iters = 10000      # total number of training iterations
batch_size = 24        # mini-batch size
block_size = 256       # block size
#####################################

# adamw optimizer
beta1 = 0.9
beta2 = 0.999

# learning rate decay settings
decay_lr = True # whether to decay the learning rate
warmup_iters = 2000 # how many steps to warm up for
lr_decay_iters = 600000 # should be ~= max_iters per Chinchilla
min_lr = 6e-5 # minimum learning rate, should be ~= learning_rate/10 per Chinchilla
grad_clip = 1.0 # clip gradients at this value, or disable if == 0.0
eval_interval = 1000
eval_iters = 200

# system
device = 'cuda' # examples: 'cpu', 'cuda', 'cuda:0', 'cuda:1' etc., or try 'mps' on macbooks
dtype = 'bfloat16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler

torch.manual_seed(1337)

n_embd = n_head * head_size

data_tr  = np.array(train_ids, dtype=np.uint16)
data_val = np.array(val_ids, dtype=np.uint16)

# poor man's data loader
def get_batch(split):
    data = data_tr if split == 'train' else data_val

    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

# init a new model from scratch
print("Initializing a new model from scratch")
conf = config(batch_size=batch_size, n_layer=n_layer, n_head=n_head, n_embd=n_embd,\
              block_size=block_size, vocab_size=vocab_size, device=device)
model = GPT(conf)

model = model.to(device)

# initialize a GradScaler. If enabled=False scaler is a no-op
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

# optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, betas=(beta1, beta2))

# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > lr_decay_iters, return min learning rate
    if it > lr_decay_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (lr_decay_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)


# training loop
X, Y = get_batch('train') # fetch the very first batch
t0 = time.time()
iter_num = 0

while True:
    # determine and set the learning rate for this iteration
    lr = get_lr(iter_num) if decay_lr else learning_rate

    logits, loss = model(X, Y)
    X, Y = get_batch('train')

    # backward pass, with gradient scaling if training in fp16
    scaler.scale(loss).backward()
    # clip the gradient
    if grad_clip != 0.0:
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), grad_clip)
    # step the optimizer and scaler if training in fp16
    scaler.step(optimizer)
    scaler.update()
    # flush the gradients as soon as we can, no need for this memory anymore
    optimizer.zero_grad(set_to_none=True)

    # evaluate the loss on train/val sets and write checkpoints
    if iter_num % eval_interval == 0:
      # timing and logging
      t1 = time.time()
      dt = t1 - t0
      t0 = t1

      losses = estimate_loss()
      print(f"step {iter_num}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f} (time lapses {dt:.4f} seconds)")

    iter_num += 1

    # termination conditions
    if iter_num > max_iters:
        break


Initializing a new model from scratch
number of parameters: 21.29M


<ipython-input-9-7df8a51bfd57>:71: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))


step 0: train loss 5.0748, val loss 5.0460 (time lapses 1.1134 seconds)
step 1000: train loss 1.8632, val loss 2.0615 (time lapses 283.2819 seconds)
step 2000: train loss 1.2955, val loss 2.0407 (time lapses 282.0643 seconds)
step 3000: train loss 0.1965, val loss 4.0200 (time lapses 280.5803 seconds)
step 4000: train loss 0.1178, val loss 4.8693 (time lapses 280.0938 seconds)
step 5000: train loss 0.1035, val loss 5.2041 (time lapses 281.1528 seconds)
step 6000: train loss 0.0954, val loss 5.4100 (time lapses 280.7283 seconds)
step 7000: train loss 0.0848, val loss 5.4946 (time lapses 281.7341 seconds)
step 8000: train loss 0.0838, val loss 5.6617 (time lapses 280.8972 seconds)
step 9000: train loss 0.0831, val loss 5.7169 (time lapses 281.8602 seconds)
step 10000: train loss 0.0708, val loss 5.9187 (time lapses 282.3412 seconds)


In [11]:
# sample from a trained model to generate new text
# (borrowed from nanoGPT, https://github.com/karpathy/nanoGPT)

import torch

# start = "\n"     # specify a text string as prompt
start = "FILE:tinyQA/prompts.txt"  #can also specify a file of multiple prompts, use as: "FILE:prompt.txt"

num_samples = 5 # number of samples to draw
max_new_tokens = 100 # number of tokens generated in each sample
temperature = 0.7 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = None #200 # retain only the top_k most likely tokens, clamp others to have 0 probability

seed = 1337
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

model.eval()
model.to(device)

# sampling the model based on one prompt
def SampleOnePrompt(prompt):
    start_ids = encode(prompt)
    x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])

    for k in range(num_samples):
        y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
        res = decode(y[0].tolist())
        answers = res.splitlines()
        print(answers[0])              # only show the first line for QA

        # sample = decode(y[0].tolist())
        # sampleAns = sample[:sample.index(']')+1]
        # print(sampleAns)    # show all sampling results
        print('---------------')


# encode the beginning of the prompt
if start.startswith('FILE:'):     # for multiple prompts
    with open(start[5:], 'r', encoding='utf-8') as f:
        Lines = f.readlines()
        for prompt in Lines:
            SampleOnePrompt(prompt[:-1])  # chop the trailing newline char
else:
     SampleOnePrompt(start)     # for one prompt


which football club did alan sugar own [ tottenham hotspur f.c. ]
---------------
which football club did alan sugar own [ tottenham hotspur f.c. ]
---------------
which football club did alan sugar own [ tottenham hotspur f.c. ]
---------------
which football club did alan sugar own [ tottenham hotspur f.c. ]
---------------
which football club did alan sugar own [ tottenham hotspur f.c. ]
---------------
 'the black and gold' is a nickname of which american football team [ steve cross ]
---------------
 'the black and gold' is a nickname of which american football team [ aria ]
---------------
 'the black and gold' is a nickname of which american football team [ steve cross ]
---------------
 'the black and gold' is a nickname of which american football team [ suday ]
---------------
 'the black and gold' is a nickname of which american football team [ are ]
---------------
alex band and aaron kamin make up which band [ the calling ]
---------------
alex band and aaron kamin make up 